##### Copyright 2025 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Translate a public domain book

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/Translate_a_Public_Domain_Book.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

In this notebook, you will explore Gemini model as a translation tool, demonstrating how to prepare data, create effective prompts, and save results into a `.txt` file.

This approach significantly improves knowledge accessibility. Various sources provide open-source books. In this notebook, you will use [Project Gutenberg](https://www.gutenberg.org/) as a resource.


This platform provides access to a wide range of books available for free download in PDF, eBook, TXT formats and more.

In [1]:
%pip install -U -q "google-genai>=1.0.0" tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.3/196.3 kB 4.8 MB/s eta 0:00:00


In [2]:
from tqdm import tqdm

from IPython.display import Markdown

## Configure your API key

To run the following cell, your API key must be stored in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) for an example.


In [3]:
from google import genai
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

client = genai.Client(api_key=GOOGLE_API_KEY)

## Data preparation

You will translate a well-known book written by Arthur Conan Doyle from Polish (trans. Eugenia Żmijewska) to English about the detective Sherlock Holmes. Here are the titles in Polish and English:



* Polish Title: *Tajemnica Baskerville'ów: dziwne przygody Sherlocka Holmes*

* English Title: *The Hound of the Baskervilles*

In [4]:
!curl https://www.gutenberg.org/cache/epub/34079/pg34079.txt > Sherlock.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  277k  100  277k    0     0   297k      0 --:--:-- --:--:-- --:--:--  296k


In [5]:
with open("/content/Sherlock.txt") as f:
      book = f.read()

Books contain all sorts of fictional or historical descriptions, some of them rather literal and might cause the model to stop from performing translation query. To prevent some of those exceptions users are able to change `safety_setting` from default to more open approach.

In [6]:
from google.genai import types
safety_settings =[
            types.SafetySetting(
              category=types.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
              threshold=types.HarmBlockThreshold.BLOCK_NONE,
            ),
            types.SafetySetting(
              category=types.HarmCategory.HARM_CATEGORY_HARASSMENT,
              threshold=types.HarmBlockThreshold.BLOCK_NONE,
            ),
            types.SafetySetting(
              category=types.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
              threshold=types.HarmBlockThreshold.BLOCK_NONE,
            ),
            types.SafetySetting(
              category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
              threshold=types.HarmBlockThreshold.BLOCK_NONE,
            )
        ]

Go ahead and initialize the Gemini Flash model for this task.

In [7]:
MODEL_ID="gemini-2.0-flash" # @param ["gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.5-flash-preview-05-20","gemini-2.5-pro-preview-05-06"] {"allow-input":true, isTemplate: true}


def generate_output(prompt):
    response = client.models.generate_content(
        model=MODEL_ID,
        contents=prompt,
        config=types.GenerateContentConfig(
          safety_settings=safety_settings
        )

    )

    try:
        return response.text
    except Exception as ex:
        raise ex

Begin by counting how many tokens are in the book to know if you need to proceed with the next step (split text into smaller chunks).

## Token Information

LLMs are constrained by token limits. In order to stay within the limit of Gemini Flash, it's important to ensure the following:

* Input token limit should not exceed: 1,048,576 tokens
* Output token limit should not exceed: 8,192 tokens

Since input chunks of post-translation text will serve as outputs, it's advisable to split them into smaller segments.

As you will see in the example, there are over 1000 small chunks. Calling API for the token count of each one would take a long time. Because of this estimate 1 token is equal to 4 characters will be used.

To account for the possible miscalculation of tokens and output potentially being longer than input each chunk will use up to 4*5000 characters. Remember to adjust the maximum to your needs.

For more details on tokens in Gemini models, refer to the documentation [here](https://ai.google.dev/gemini-api/docs/models/gemini
).

### Split text into chunks
Adjust the number of whitespace characters used for splitting based on the book you are working with.

Then, ensure to consider token limitations to receive complete responses from the model.


In [8]:
chunks = book[:book.find("END OF THE PROJECT GUTENBERG EBOOK TAJEMNICA BASKERVILLE'ÓW: DZIWNE PRZYGODY SHERLOCKA HOLMES")].split("\n\n")

num_chunks = len(chunks)
print(f"Number of chunks: {num_chunks}")

Number of chunks: 1802


In [9]:
estimated_token_counts = []

for chunk in chunks:
    if not chunk:  # Check if chunk is empty or None
        continue  # Skip this iteration if chunk is empty

    # Process non-empty chunk
    estimated_token_count = len(chunk)/4
    estimated_token_counts.append(estimated_token_count)

# You can print number of estimated tokens in each non empty chunk to see what are you working with
print(estimated_token_counts)

[134.5, 16.25, 6.5, 7.25, 25.0, 4.0, 58.25, 26.0, 50.25, 148.0, 10.25, 11.0, 7.5, 8.5, 10.0, 9.5, 9.25, 7.75, 9.25, 9.5, 7.25, 11.25, 0.75, 5.0, 136.75, 22.25, 37.0, 12.5, 73.25, 45.0, 11.5, 24.25, 8.5, 45.5, 11.75, 31.75, 50.0, 48.0, 45.25, 22.0, 38.25, 52.25, 6.75, 6.25, 3.75, 54.25, 19.5, 15.75, 24.25, 45.25, 4.5, 152.75, 22.0, 72.0, 135.0, 91.5, 9.75, 53.5, 38.0, 5.25, 120.0, 128.0, 37.5, 9.25, 3.5, 10.25, 15.75, 11.25, 15.5, 27.75, 31.25, 16.5, 13.75, 10.25, 73.25, 44.5, 66.0, 13.5, 28.0, 11.25, 43.0, 33.5, 1.0, 7.5, 14.5, 17.5, 6.25, 5.5, 4.0, 78.5, 16.5, 9.0, 111.0, 17.75, 35.75, 34.5, 15.5, 21.0, 30.5, 57.5, 33.75, 21.75, 158.75, 60.5, 39.0, 110.0, 125.75, 124.5, 66.0, 55.75, 40.25, 63.0, 54.5, 84.75, 45.0, 33.5, 90.5, 138.25, 37.75, 10.0, 38.75, 5.5, 10.25, 12.25, 10.0, 66.0, 19.5, 125.75, 62.5, 50.25, 59.25, 44.5, 131.25, 65.75, 20.25, 9.0, 65.5, 32.0, 50.75, 52.75, 68.5, 60.25, 48.25, 54.25, 10.0, 15.5, 8.5, 92.75, 1.75, 17.0, 24.25, 150.75, 29.75, 106.75, 103.75, 76.25, 59.

Identify the largest number of tokens in a chunk to check if any exceed your token limitations. In this case, the chunks are very small, so this should not be an issue.

If you encounter this problem, consider splitting the text into even smaller chunks.

In [10]:
max(estimated_token_counts)

272.75

### Group smaller chunks together
As observed above, certain chunks contain very few tokens. While the LLM can translate these sentences accurately, providing larger context can enhance its understanding of the content.

Let's combine smaller chunks into larger ones for improved results.

In [11]:
# Combine short chunks into larger groups based on token counts.
def chunk_grouping(chunks, token_counts, max_len=5000):
    grouped_chunks = []
    current_group = ""
    current_token_sum = 0

    # Process each chunk and group them based on token limits
    for chunk, count in zip(chunks, token_counts):
        # Skip chunks that exceed the max token limit
        if count > max_len:
            continue

        # Add a new chunk if there is space available in the current group.
        if current_token_sum + 1 + count <= max_len:
            current_group += "\n\n" + chunk
            current_token_sum += 1 + count  # Count in 1 token for newlines

        # If adding this chunk exceeds the current group's capacity, start a new group
        else:
            grouped_chunks.append(current_group)
            current_group = chunk
            current_token_sum = count

    if current_group:  # Add the last remaining group
        grouped_chunks.append(current_group)

    return grouped_chunks


chunks = chunk_grouping(chunks, estimated_token_counts)
print(len(chunks))

13


The last step of splitting is checking if the non-estimated token counts are acceptable.

In [12]:
token_counts = []
for chunk in chunks:
  response = client.models.count_tokens(
      model = MODEL_ID,
      contents = chunk,
  )
  token_count = response.total_tokens
  token_counts.append(token_count)
token_counts

[6309, 6465, 6603, 6371, 6761, 6534, 6271, 7111, 6392, 6718, 6821, 5670, 3242]

Despite using estimates, this split minimizes the calls necessary to translate the book well, and most chunks are of similar token counts to each other.

## Translation

Your book is ready for translations! The selected book has been divided into 13 larger chunks.

### Prompt

Specify the original language of your book and the language you would like it to be translated into.


In [13]:
def translations(text,
                 inputLanguage='Polish',
                 targetLanguage='English'):

    translation_prompt = f"""
      As a professional book translator,
      translate the following book from {inputLanguage} into {targetLanguage}.

      Book to Translate:
      {text}

    """
    return translation_prompt

### Translate chunks

The translation time depends on the amount of text being processed. Our selected book was 318 KB and the entire translation took 20 minutes.

Please note that larger texts may take several hours to complete.

In [14]:
results = []
index = 0
translation = None

for chunk in tqdm(chunks):
    try:
        translation = generate_output(translations(chunk))
        results.append(translation)
    except Exception as ex:
        print(f"Chunk was not included in translation due to possible sensitive content. Text: {chunk}")

    index += 1

 23%|██▎       | 3/13 [01:17<04:19, 25.98s/it]


KeyboardInterrupt: 

All done, now you can print the chunk before and after translation to see the results.

You can see, that translation effectively maintains the narrative's novel quality, proper grammar and sentence structure.

Next step will be joining chunks together and saving the translated book.

In [15]:
Markdown(chunks[1])

Doktor Mortimer skończył czytać tę niezwykłą opowieść, nasunął na
oczy okulary i spojrzał na pana Sherlocka Holmes. Ten ziewnął
i rzucił papierosa w ogień.

-- No i cóż? -- rzekł.

-- Czy historya ta nie jest interesującą?

-- Zapewne, dla zbieracza starych legend i baśni.

Doktor Mortimer wyjął z kieszeni gazetę.

-- A teraz, panie Holmes -- rzekł -- pokażę panu coś aktualniejszego.
Oto dziennik „Devon County Chronicle” z dnia 14-go Maja roku
bieżącego. Jest w nim opis faktów, odnoszących się do śmierci sir
Karola Baskerville, która zdarzyła się na kilka dni przed ową datą.

Mój przyjaciel nachylił się i począł słuchać uważniej. Doktor
Mortimer czytał:

„Niedawna i nagła śmierć sir Karola Baskerville, kandydata liberalnego
z hrabstwa Devon, przejęła całą naszą okolicę zdumieniem i trwogą.
Jakkolwiek sir Karol Baskerville mieszkał w Baskerville Hall od
niedawna, lecz jego uprzejmość, hojność i szlachetność, zjednały mu
szacunek tych wszystkich, którzy mieli z nim do czynienia.
W dzisiejszej epoce parweniuszów miło jest widzieć potomka
starożytnego a podupadłego rodu, odzyskującego fortunę swych
przodków i przywracającego świetność staremu nazwisku.

„Sir Karol, jak wiadomo, zyskał duży majątek na spekulacyach w Afryce
południowej, a będąc z natury przezornym, nie kusił dalej szczęścia,
które mogłoby się odwrócić od niego, jak to czyni z innymi,
i zrealizowawszy swoją fortunę, powrócił do Anglii.

„Przed dwoma laty zaledwie osiadł w Baskerville Hall; znane są jego
szerokie plany odnowienia starej siedziby i wprowadzenia ulepszeń
w gospodarstwie rolnem. Śmierć przeszkodziła ich urzeczywistnieniu.

„Będąc bezdzietnym, pragnął, aby cała okolica korzystała z jego
fortuny i niejeden z sąsiadów ma ważne powody do opłakiwania jego
przedwczesnego zgonu. Donosiliśmy często na tych szpaltach o jego
hojnych ofiarach na instytucye publiczne.

„Śledztwo nie wyjaśniło dotychczas okoliczności, towarzyszących
śmierci sir Karola. Krążą tu najdziwaczniejsze legendy, przypisujące
ów nagły zgon działaniu sił nadprzyrodzonych.

„Sir Karol był wdowcem, uważano go powszechnie za dziwaka. Pomimo
dużej fortuny, miał przyzwyczajenia proste, skromne. Personel służbowy
w Baskerville Hall składał się z kamerdynera, nazwiskiem Barrymore,
jego żony, kucharki i gospodyni w jednej osobie. Oboje zeznają, że
zdrowie sir Karola w ostatnich czasach było nietęgie, że rozwijała się
w nim choroba serca, objawiająca się bladością, brakiem tchu
i częstemi omdleniami. Doktor Jerzy Mortimer, przyjaciel i domowy lekarz
nieboszczyka, złożył zeznanie w tym samym duchu.

„Sir Karol Baskerville zwykł był co wieczór, przed pójściem na
spoczynek, przechadzać się po słynnej alei wiązów przed zamkiem.
Małżonkowie Barrymore opowiadają, że wieczorom 4-go maja, ich pan
oznajmił im, że nazajutrz wyrusza do Londynu i kazał pakować kuferek.

„Owego wieczoru wyszedł, jak zwykle, na przechadzkę, wśród której
wypalał cygaro.

„Z tej przechadzki już nie powrócił.

„O dwunastej Barrymore, widząc drzwi frontowe jeszcze otwarte,
zaniepokoił się, i wziąwszy latarnię, poszedł szukać swego pana.
W ciągu dnia deszcz padał, więc łatwo było dojrzeć ślady stóp sir
Karola wzdłuż alei. W połowie drogi jest furtka, prowadząca na łąkę.

„Były ślady, że sir Karol stał przy niej przez czas pewien,
następnie skręcił znowu w aleję. Znaleziono jego trupa na jej końcu.

„Pomiędzy innemi dotychczas nie wyjaśniono jednego faktu, wynikającego
z zeznań Barrymora, a mianowicie, że ślady kroków jego pana zmieniły
się z chwilą, gdy stał przy furtce i że odtąd szedł na palcach.

„Niejaki Murphy, cygan, handlujący końmi, znajdował się wówczas na
łące, w pewnej odległości od nieboszczyka, ale sam przyznaje, że był
pijany. Powiada, że słyszał krzyki, ale nie może określić, skąd
wychodziły.

„Na ciele sir Karola nie było śladów przemocy lub gwałtu, a chociaż
lekarz zeznaje, że wyraz twarzy nieboszczyka był tak zmieniony, że on,
doktor Mortimer, w pierwszej chwili poznać go nie mógł, rzeczoznawcy
tłómaczą to anewryzmem serca, przy którym zachodzą podobne objawy.

„Sekcya wykazała przedawnioną wadę serca, sędzia śledczy potwierdził
badanie lekarskie. Pożądanem jest, aby spadkobierca sir Karola mógł
objąć jaknajprędzej majątek i prowadzić dalej chlubną działalność,
przerwaną tak nagle i tak tragicznie.

„Gdyby prozaiczne orzeczenie sędziego śledczego i rzeczoznawców nie
położyło końca romantycznym historyom, krążącym na temat owej
śmierci, nie łatwo byłoby znaleźć właściciela Baskerville Hall.

„Domniemanym spadkobiercą jest pan Henryk Baskerville, syn młodszego
brata sir Karola. Ów młodzieniec przed kilku laty wyruszył do Ameryki.
Zarządzono poszukiwania. Dowie się on zapewne niebawem o zmianie swego
losu”.

       *       *       *       *       *

Doktor Mortimer złożył gazetę i wsunął ją do kieszeni, mówiąc:

-- To są fakty, powszechnie znane.

-- Dziękuję panu za zwrócenie mojej uwagi na sprawę istotnie 
ciekawą -- rzekł pan Holmes. -- Czytałem o niej, coprawda, w dziennikach, 
ale w owym czasie byłem zajęty kameami, które zniknęły z muzeum
Watykańskiego. Chciałem się przysłużyć Papieżowi i straciłem z oczu
to, co się jednocześnie działo w Anglii. Ten artykuł, powiadasz pan,
zawiera fakty, znane powszechnie?

-- Tak.

-- Zechciej więc pan uwiadomić mnie teraz o tem, czego nikt nie wie.

Sherlock Holmes oparł się o poręcz fotelu i znowu złożył ręce tak,
aby palce stykały się końcami.

-- Uczynię to -- rzekł doktor Mortimer, zdradzając coraz większe
wzburzenie. -- Powiem panu to, czego nie mówiłem jeszcze nikomu.
Zataiłem to przed sędzią śledczym z pobudek, które pan zapewne
zrozumie. Jako człowiek nauki, nie chciałem zdradzać się publicznie,
iż wierzę w przesądy. Dalej, rozumiałem, że gdyby utwierdziła się
wiara w nadprzyrodzone siły, rządzące w Baskerville Hall, nikt nie
zechciałby objąć starego zamku w posiadanie. Dla tych dwóch powodów
nie zeznałem przed sądem wszystkiego, co wiem -- albowiem nie
zdałoby się to na nic sędziemu śledczemu -- ale z panem będę
zupełnie szczery.

„Okolica jest mało zaludniona, rezydencye rzadkie, a siedziby
włościańskie rozrzucone na znacznej odległości od siebie.

„Widywałem często sir Karola Baskerville, który był spragniony
towarzystwa. Oprócz p. Frankland w Lafterhall i naturalisty,
p. Stapleton, w pobliżu niema ludzi inteligentnych. Sir Karol
był skryty, małomówny. Zbliżyliśmy się z powodu jego choroby,
przytem łączyły nas wspólne upodobania naukowe. Przywiózł wiele
ciekawych wiadomości z Afryki Południowej. Spędziliśmy niejeden
miły wieczór na rozprawach o anatomii porównawczej.

„W ciągu ostatnich paru miesięcy spostrzegłem, że system nerwowy sir
Karola był nadwerężony. Wziął tak dalece ową legendę do serca, że
choć co wieczór odbywał spacery, nic go nie mogło skłonić do
wejścia na łąkę po zachodzie słońca. Prześladowała go wciąż obawa
upiorów, i pytał mnie nieraz, czym nie widział jakiego dziwacznego
stworzenia i czym nie słyszał szczekania. Te pytania zadawał mi zawsze
głosem drżącym.

„Wraziła mi się w pamięć jedna z moich wizyt u niego przed trzema
tygodniami. Stał w sieni. Gdym schodził z wózka, spostrzegłem, że
patrzy po przez moje ramię ze strachem w oczach. Odwróciłem się
i spostrzegłem jakiś kształt dziwny, podobny do dużego, czarnego
cielęcia. Przeleciało to po za moim wózkiem.

„Sir Karol był tak wzburzony, że poszedłem szukać tego osobliwego
zwierzęcia. Ale nigdzie go nie było. Mój pacyent nie mógł się
uspokoić. Zostałem z nim przez cały wieczór i wtedy, tłómacząc swe
wzburzenie, opowiedział mi całą tę historyę.

„Wspominam o tym drobnym epizodzie, gdyż nabiera on znaczenia wobec
tragedyi, która potem nastąpiła; na razie nie przywiązywałem do tego
wagi i dziwiłem się wzburzeniu sir Karola.

„Miał jechać do Londynu z mojej porady. Wiedziałem, że jest chory
na serce i że ciągły niepokój źle wpływa na jego zdrowie. Sądziłem,
że parę miesięcy, spędzonych na rozrywkach, uwolni go od tych mar
i przywidzeń. Nasz wspólny przyjaciel, pan Stapleton, był tego samego
zdania. I oto wynikło nieszczęście.

„Zaraz po śmierci sir Karola, kamerdyner Barrymore wyprawił do mnie
grooma Perkinsa; przybyłem do Baskerville Hall w godzinę po
katastrofie. Zbadałem wszystkie fakty, przytoczone w śledztwie.
Szedłem śladami, pozostawionemi w Alei Wiązów, obejrzałem miejsce przy
furtce, gdzie sir Karol zatrzymał się, i zmiarkowałem, że od owego
miejsca szedł na palcach i że nie było śladu innych kroków, oprócz
późniejszych, Barrymora. Wreszcie obejrzałem starannie zwłoki, które
pozostały nietknięte do mego przybycia.

„Sir Karol leżał nawznak z rozciągniętemi rękoma, jego twarz była
tak zmieniona, że zaledwie mogłem ją poznać. Na ciele nie było
żadnych obrażeń. Ale Barrymore w toku śledztwa uczynił jedno zeznanie
fałszywe. Powiedział, że nie było żadnych śladów dokoła trupa. Nie
spostrzegł ich, ale ja dostrzegłem -- zupełnie świeże, w pobliżu.

-- Czy ślady kroków?

-- Tak.

-- Męzkie, czy kobiece?

Doktor Mortimer patrzył na nas przez chwilę dziwnemi oczyma, wreszcie
odparł szeptem:

-- Panie Holmes, były ślady kroków... psa... olbrzymiego.




III.

Zagadka.


Przyznaję, że gdym słuchał tego opowiadania, przebiegały po mnie
dreszcze. I doktor był żywo poruszony.

-- Widziałeś pan te ślady? -- spytałem.

-- Tak, na własne oczy; tak wyraźnie, jak teraz widzę pana.

-- I nic pan nie mówiłeś?

-- Po co?

-- Dlaczego nikt inny nie dostrzegł tych śladów?

-- Pozostały o jakie dwa łokcie od trupa; nie zwrócono na nie uwagi
i jabym ich nie zauważył, gdybym nie był znał tej legendy.

-- Dużo jest psów w okolicy.

-- Tak, ale to nie był zwykły pies. Powiedziałem już panom, że był
ogromny.

-- Czy nie zbliżył się do trupa?

-- Nie.

-- Czy noc była jasna?

-- Nie; pochmurna i wilgotna.

-- Ale deszcz nie padał?

-- Nie.

-- Jak wygląda aleja?

-- Jest wysadzana dwoma rzędami wiązów i opasana żywopłotem,
wysokości 12 stóp. Sama aleja ma 8 stóp szerokości.

-- Czy jest co pomiędzy żywopłotem a aleją?

-- Tak; po obu stronach biegnie trawnik, szerokości 6 stóp.

-- O ile zrozumiałem, jest wejście przez furtkę?

-- Tak; furtka prowadzi na łąkę.

-- Czy są inne wejścia?

-- Niema.

-- Tak, iż aby wkroczyć w Aleję Wiązów, trzeba wyjść z domu, lub
iść przez łąkę?

-- Jest trzecie wejście przez altanę, na drugim końcu alei.

-- Czy sir Karol doszedł do tej altany?

-- Nie; znaleziono go o pięćdziesiąt łokci od niej.

-- A teraz, zechciej mi pan powiedzieć, doktorze Mortimer -- będzie to
szczegół ważny -- czy ślady, któreś pan spostrzegł, pozostały na
żwirze, czy na trawie?

-- Nie mogłem dojrzeć śladów na trawie.

-- Czy ślady były po tej samej stronie, co furtka?

-- Tak, po tej samej.

-- Bardzo mnie pan zaciekawia. Jeszcze jedno pytanie. Czy furtka była
zamknięta?

-- Zamknięta na klucz i zaryglowana.

-- Jak jest wysoka?

-- Ma około 4-ch stóp.

-- A więc można ją łatwo przesadzić?

-- Tak.

-- A jakie ślady znalazłeś pan przy furtce?

-- Żadnych, któreby mogły zwrócić uwagę.

-- Czyżeś pan nie oglądał ziemi dokoła?

-- I owszem, oglądałem.

-- I nie dostrzegłeś pan żadnych śladów?

-- Były bardzo niewyraźne. Widocznie sir Karol stał tutaj przez pięć
do dziesięciu minut.

-- Skąd pan to wie?

-- Gdyż popiół z cygara spadł dwa razy na ziemię.

-- Wybornie. Znaleźliśmy kolegę wedle naszego serca. Prawda, Watson?
Ale jakież to były ślady?

-- Ślady stóp na żwirze. Innych znaków dostrzedz nie mogłem.

Sherlock Holmes uderzył ręką w kolano.

-- Czemu mnie tam nie było! -- zawołał. -- Jest to wypadek bardzo
ciekawy, dający obfite pole do naukowej ekspertyzy. Czemuż nie mogłem
odczytać tej żwirowej karty, zanim ją zatarły inne stopy! Doktorze
Mortimer, że też mnie pan nie uwiadomił wcześniej! Jeśli nam się nie
uda wyświetlić sprawy, cała odpowiedzialność spadnie na pana.

-- Nie mogłem pana wzywać, panie Holmes, bo w takim razie musiałbym
ujawnić te fakty przed całym światem, a mówiłem już, żem sobie tego
nie życzył. Zresztą... zresztą...

-- Czemu pan nie kończysz?

-- Bywają dziedziny, niedostępne nawet dla najbystrzejszych
i najdoświadczeńszych detektywów...

-- Chcesz pan powiedzieć, że wchodzi to w zakres rzeczy
nadprzyrodzonych?

-- Tego nie twierdzę stanowczo.

-- Ale pan tak myślisz w głębi ducha.

-- Od owej tragedyi doszły do mojej świadomości fakty, sprzeczne
z ogólnemi prawami natury.

-- Naprzykład?

-- Dowiaduję się, że przed tą katastrofą, kilku ludzi widziało na
łące niezwykłe stworzenie, podobne do złego ducha Baskervillów.
Wszyscy mówią, że był to pies ogromny, przezroczysty. Wypytywałem
tych ludzi -- jeden z nich jest włościaninem, drugi kowalem,
trzeci farmerem. Ich zeznania są jednakowe. W całej okolicy
zapanował strach przesądny. Nikt po nocy nie przejdzie przez łąkę.

-- I pan, człowiek nauki, wierzy w takie baśnie? -- zawołał Holmes.

-- Dotychczas moje badania ogarniały świat zmysłów; usiłowałem
walczyć z chorobą, ale ze złymi duchami walczyć nie umiem. Zresztą
musisz pan przyznać, że ślady stóp są dowodem materyalnym. Pies
Hugona nie był także zjawiskiem nadprzyrodzonem skoro mógł zagryźć
na śmierć, a jednak miał w sobie coś szatańskiego.

-- Widzę, że pan przeszedłeś do obozu spirytystów. Ale zechciej mi
powiedzieć jeszcze jedno, doktorze Mortimer. Jeżeli pan skłaniasz się
ku takim zapatrywaniom, dlaczego przyszedłeś zasięgnąć mojej rady?
Powiadasz pan jednym tchem, że nie warto przeprowadzać śledztwa
w sprawie zabójstwa sir Karola, a jednocześnie prosisz mnie, żebym się
tą sprawą zajął.

-- Nie prosiłem o to.

-- Więc w jaki sposób mogę panu dopomódz?

-- Radząc mi, co mam zrobić z sir Henrykiem Baskerville, który
przybywa na dworzec Waterloo -- doktor Mortimer spojrzał na zegarek --
za godzinę i kwadrans -- dokończył.

-- Czy on jest spadkobiercą?

-- Tak. Po śmierci sir Karola zasięgaliśmy wiadomości o tym
gentlemanie i dowiedzieliśmy się, że ma fermę w Kanadzie. Wedle
naszych informacyj, jest to młodzieniec bez zarzutu. Nie mówię teraz
jako lekarz, lecz jako wykonawca testamentu sir Karola.

-- Czy niema innych kandydatów do spadku?

-- Żadnego. Mógłby nim być tylko Roger Baskerville, najmłodszy
z trzech braci, z których sir Karol był najstarszym. Drugi brat, zmarły
przedwcześnie, był właśnie ojcem owego Henryka. Trzeci, Roger, był
synem marnotrawnym, żywą podobizną duchową starego Hugona. Tyle
nabroił w Anglii, że nie mógł już tu przebywać, uciekł do Ameryki
środkowej i umarł w roku 1876-ym na żółtą febrę. Henryk jest
ostatnim z Baskervillów. Za godzinę i pięć minut mam go spotkać
na dworcu Waterloo. Miałem depeszę z uwiadomieniem, iż przybył dziś
do Southampton. A teraz, panie Holmes, powiedz, jak mi radzisz postąpić?

-- Dlaczego sir Henryk nie miałby wrócić do domu swych ojców?

-- Wydaje się to rzeczą naturalną, a jednak, jeśli weźmiemy pod
uwagę, że każdego z Baskervillów, który tam przebywa, czeka śmierć
nagła i gwałtowna... Jestem pewien, że gdyby sir Karol mógł był ze
mną mówić przed katastrofą, byłby mi zakazał wprowadzać ostatniego
z rodu do owej przeklętej rezydencyi. Z drugiej strony, dobrobyt
całej okolicy zależy od przebywania dziedzica w Baskerville Hall.
Całe dzieło, zapoczątkowane przez sir Karola, pójdzie w niwecz,
jeśli nikt nie zamieszka na zamku. Boję się, aby dobro tej okolicy
nie skłoniło mnie do nielojalnego postąpienia z sir Henrykiem
i dlatego proszę pana o radę.

Holmes zastanawiał się długą chwilę.

-- Zatem -- odezwał się -- według pańskiego przekonania, jakaś siła
nieczysta grozi w tych stronach Baskervillom. Wszak pan w to wierzy
święcie?

-- Gotów jestem przypuszczać, że tak jest.

-- W takim razie, ta siła nieczysta może pastwić się nad Baskervillem
równie dobrze w Londynie, jak i w Devonshire. Szatan, którego
działanie byłoby umiejscowione, nie byłby groźnym.

-- Starasz się pan ośmieszyć moją obawę, panie Holmes. Lecz gdybyś
sam widział te rzeczy nadprzyrodzone, odechciałoby ci się żartów.
Z pańskich słów miarkuję, że ów młodzieniec jest równie bezpieczny
w Londynie, jak i w Devonshire. Przybywa za pięćdziesiąt minut. Cóż
mi pan radzisz?

-- Radzę wziąć dorożkę, zawołać psa, który skowyczy za drzwiami
i jechać na dworzec Waterloo na spotkanie sir Henryka Baskerville.

-- A potem?

-- A potem nic mu pan nie powiesz, dopóki nie namyślę się w tym
względzie.

-- Jak długo potrzebujesz pan namyślać się?

-- Dwadzieścia cztery godziny. Poproszę cię, doktorze Mortimer, abyś
mnie odwiedził jutro rano o dziesiątej. A zechciej przywieźć ze sobą
sir Henryka Baskerville. To mi ułatwi wykonanie mego planu.

-- Zrobię, jak pan chcesz.

Doktor zapisał godzinę na mankiecie i wyszedł. Pan Holmes zatrzymał go
na schodach.

-- Jeszcze jedno pytanie -- rzekł. -- Powiadasz pan, że przed śmiercią
sir Karola kilku ludzi widziało to zjawisko na łące?

-- Tak, trzech ludzi.

-- Czy który z nich widział je potem?

-- Nie wiem.

-- Dziękuję panu. Dowidzenia.

Holmes powrócił na swoje miejsce. Był widocznie zadowolony.

-- Wychodzisz, Watson? -- rzekł.

-- Czy potrzebujesz mojej pomocy?

-- Nie, mój drogi. Poproszę cię o nią dopiero w chwili działania.
Sprawa wyjątkowa. Przechodząc obok Bradleya, zechciej wstąpić do
sklepu i każ mi przynieść paczkę najmocniejszego tytoniu. Jeżeli
możesz, byłbym ci bardzo obowiązany, gdybyś tu nie wracał przed
wieczorem, a wtedy zestawimy nasze wrażenia i poglądy.

Wiedziałem, że samotność jest niezbędną mojemu przyjacielowi
w chwilach, gdy zastanawiał się nad poszlakami spraw kryminalnych, gdy
wyciągał wnioski i tworzył teorye, które okazywały się zawsze
słusznemi. To też cały dzień spędziłem w klubie i dopiero około
dziewiątej powróciłem do mieszkania przy Bakerstreet.

Gdy drzwi otworzyłem, zdało mi się, że w mieszkaniu był pożar;
światło lampy ukazywało się jakby za czarną mgłą. Po chwili
zmiarkowałem, że dym pochodzi nie od ognia, lecz od mocnego tytoniu.
Wśród kłębów ujrzałem Holmesa w szlafroku. Siedział w fotelu
z fajeczką w ustach. Na stole leżało kilka zwitków papieru.
Odkaszlnąłem.

-- Zaziębiłeś się? -- spytał.

-- Nie, ale można się tu udusić.

-- Otwórz okno. Widzę, że spędziłeś cały dzień w klubie...

-- Po czem to miarkujesz, Holmes?

-- Jesteś rzeźwy, pachnący, w dobrym humorze. Nigdy nie domyślisz się,
gdzie ja byłem.

-- Nie będę nad tem suszył głowy. Powiesz mi sam.

-- A więc byłem w Devonshire.

-- Myślą?

-- Tak. Moje ciało pozostało tutaj, na tym fotelu, i skonsumowało dwa
olbrzymie imbryki kawy i niezliczoną moc tytoniu. Po twojem wyjściu
posłałem do Stamforda po mapę tej okolicy i błądziłem po niej przez
dzień cały. Pochlebiam sobie, że każda piędź ziemi jest mi teraz
dobrze znana.

-- To zapewne mapa o wielkiej skali?

-- Tak.

Rozwinął ją i położył na kolanie.

-- Widzisz -- mówił -- oto łąka, a to -- Baskerville Hall.

-- Naokoło las.

-- Istotnie. A oto Aleja Wiązów, na lewo od łąki. Tu jest wioska
Grimpen, w której doktor Mortimer obrał swoją główną kwaterę.
W obrębie mil pięciu mało jest ludzkich siedzib. Oto Lafter Hall.
Tu -- domek przyrodnika Stapleton. Tutaj dwie formy: High Tore
i Fulmire. O czternaście mil dalej -- więzienie państwowe Princetown.
Dokoła i pośrodku -- łąki i trzęsawiska. Taki jest teren, na którym
rozegrała się owa tragedya. Postaramy się odtworzyć wszystkie jej
sceny i akty.

-- Miejscowość bezludna.

-- Tak. Dyabeł mógł się na niej rozgościć...

-- A zatem i pan skłaniasz się do nadprzyrodzonych wyjaśnień...

In [16]:
Markdown(results[1])

## The Hound of the Baskervilles (Translated from Polish)

Dr. Mortimer finished reading this extraordinary tale, adjusted his glasses on his nose, and looked at Mr. Sherlock Holmes. The latter yawned and threw his cigarette into the fire.

“Well?” he said.

“Is this not an interesting story?”

“Doubtless, for a collector of old legends and fairy tales.”

Dr. Mortimer took a newspaper from his pocket.

“And now, Mr. Holmes,” he said, “I will show you something more current. Here is the 'Devon County Chronicle' from the 14th of May of this year. It contains a description of the facts relating to the death of Sir Charles Baskerville, which occurred a few days before that date.”

My friend leaned forward and began to listen more attentively. Dr. Mortimer read:

“The recent and sudden death of Sir Charles Baskerville, the Liberal candidate from the county of Devon, has filled our entire region with astonishment and dread. Although Sir Charles Baskerville had only resided at Baskerville Hall for a short time, his courtesy, generosity, and nobility earned him the respect of all who had dealings with him. In today’s era of parvenus, it is pleasing to see a descendant of an ancient and impoverished family recovering the fortune of his ancestors and restoring the splendor to an old name.

“Sir Charles, as is known, amassed a large fortune through speculation in South Africa, and being prudent by nature, he did not tempt fate further, which could have turned against him, as it does against others, and having realized his fortune, he returned to England.

“He had only settled in Baskerville Hall two years ago; his extensive plans to renovate the old residence and introduce improvements in agriculture are well known. Death has prevented their realization.

“Being childless, he wished the entire region to benefit from his fortune, and more than one neighbor has important reasons to mourn his untimely demise. We have frequently reported in these columns about his generous contributions to public institutions.

“The investigation has so far failed to clarify the circumstances surrounding Sir Charles’s death. The most bizarre legends are circulating here, attributing this sudden death to the action of supernatural forces.

“Sir Charles was a widower, and he was generally considered an eccentric. Despite his great wealth, he had simple, modest habits. The staff at Baskerville Hall consisted of a butler, named Barrymore, his wife, a cook and housekeeper in one. Both testify that Sir Charles's health had been failing in recent times, that he was developing heart disease, manifested by paleness, shortness of breath, and frequent fainting spells. Dr. Jerzy Mortimer, a friend and the deceased’s family doctor, testified in the same vein.

“Sir Charles Baskerville used to take a walk every evening, before going to bed, along the famous Yew Alley in front of the mansion. The Barrymores relate that on the evening of May 4th, their master informed them that he was leaving for London the next day and ordered his suitcase to be packed.

“That evening, he went out for a walk, as usual, during which he smoked a cigar.

“He never returned from that walk.

“At twelve o'clock, Barrymore, seeing the front door still open, became worried and, taking a lantern, went to look for his master. During the day, it had rained, so it was easy to see the footprints of Sir Charles along the alley. Halfway down the path there is a gate leading to the meadow.

“There were signs that Sir Charles had stood by it for some time, then turned back into the alley. His body was found at the end of the path.

“Among other things, one fact arising from Barrymore’s testimony has not yet been explained, namely, that the footprints of his master changed from the moment he stood by the gate and that from then on he walked on his toes.

“A certain Murphy, a Gypsy horse trader, was in the meadow at the time, at some distance from the deceased, but he himself admits that he was drunk. He says he heard screams, but cannot specify where they came from.

“There were no traces of violence or assault on Sir Charles’s body, and although the doctor testified that the deceased’s facial expression was so altered that he, Dr. Mortimer, could not recognize him at first, the experts attributed this to a heart aneurysm, which causes similar symptoms.

“The autopsy revealed a chronic heart defect, and the coroner confirmed the medical examination. It is desirable that Sir Charles’s heir should take possession of the estate as soon as possible and continue the commendable activities, interrupted so suddenly and tragically.

“If the prosaic verdict of the coroner and the experts had not put an end to the romantic stories circulating about this death, it would not have been easy to find an owner for Baskerville Hall.

“The presumed heir is Mr. Henry Baskerville, the son of Sir Charles’s younger brother. This young man left for America several years ago. A search has been ordered. He will probably soon learn about the change in his fortunes.”

       *       *       *       *       *

Dr. Mortimer folded the newspaper and slipped it into his pocket, saying:

“These are the facts, generally known.”

“I thank you for drawing my attention to a truly interesting case,” said Mr. Holmes. “I read about it, of course, in the newspapers, but at that time I was occupied with the cameos that disappeared from the Vatican Museum. I wanted to be of service to the Pope and lost sight of what was happening simultaneously in England. This article, you say, contains facts generally known?”

“Yes.”

“Then please inform me now of what no one knows.”

Sherlock Holmes leaned back against the armrest of the chair and again folded his hands so that his fingertips touched.

“I will do so,” said Dr. Mortimer, showing increasing agitation. “I will tell you what I have not told anyone else. I withheld it from the coroner for reasons that you will surely understand. As a man of science, I did not want to publicly reveal that I believe in superstitions. Furthermore, I understood that if the belief in supernatural forces governing Baskerville Hall was established, no one would be willing to take possession of the old mansion. For these two reasons, I did not testify before the court everything I know -- for it would not have been of any use to the coroner -- but with you I will be completely frank.

“The area is sparsely populated, the residences rare, and the peasant settlements scattered at a considerable distance from each other.

“I often saw Sir Charles Baskerville, who was eager for company. Apart from Mr. Frankland in Lafterhall and the naturalist, Mr. Stapleton, there are no intelligent people nearby. Sir Charles was secretive, taciturn. We became close because of his illness, and we also shared scientific interests. He brought back much interesting information from South Africa. We spent many pleasant evenings discussing comparative anatomy.

“Over the past few months, I noticed that Sir Charles’s nervous system was strained. He took this legend so much to heart that although he took walks every evening, nothing could persuade him to enter the meadow after sunset. He was constantly haunted by the fear of ghosts, and he often asked me if I had seen some strange creature and if I had heard barking. He always asked these questions in a trembling voice.

“One of my visits to him three weeks ago is etched in my memory. He was standing in the hall. As I got out of the carriage, I noticed that he was looking over my shoulder with fear in his eyes. I turned around and saw some strange shape, similar to a large, black calf. It flew past behind my carriage.

“Sir Charles was so agitated that I went to look for this peculiar animal. But it was nowhere to be found. My patient could not calm down. I stayed with him for the entire evening, and then, explaining his agitation, he told me the whole story.

“I mention this minor episode because it takes on significance in light of the tragedy that followed; at the time, I did not attach any importance to it and was surprised by Sir Charles’s agitation.

“He was supposed to go to London on my advice. I knew that he had heart disease and that constant anxiety was bad for his health. I thought that a few months spent on entertainment would free him from these nightmares and visions. Our mutual friend, Mr. Stapleton, was of the same opinion. And that is what resulted in this misfortune.

“Immediately after Sir Charles’s death, the butler Barrymore sent the groom Perkins to me; I arrived at Baskerville Hall an hour after the catastrophe. I examined all the facts mentioned in the investigation. I followed the traces left in Yew Alley, looked at the place by the gate where Sir Charles stopped, and noticed that from that place he walked on his toes and that there was no trace of other steps, apart from Barrymore’s later ones. Finally, I carefully examined the corpse, which remained untouched until my arrival.

“Sir Charles lay on his back with outstretched arms, his face so altered that I could barely recognize him. There were no injuries on the body. But Barrymore made one false statement during the investigation. He said that there were no traces around the corpse. He did not notice them, but I did -- completely fresh, nearby.”

“Footprints?”

“Yes.”

“Male or female?”

Dr. Mortimer looked at us for a moment with strange eyes, finally replying in a whisper:

“Mr. Holmes, there were footprints… of a dog… a gigantic one.”

III.

The Riddle.

I confess that as I listened to this story, shivers ran down my spine. And the doctor was visibly moved.

“You saw these footprints?” I asked.

“Yes, with my own eyes; as clearly as I see you now.”

“And you said nothing?”

“What for?”

“Why did no one else see these tracks?”

“They were about two cubits from the corpse; no attention was paid to them, and I would not have noticed them if I had not known the legend.”

“There are many dogs in the area.”

“Yes, but this was no ordinary dog. I have already told you that it was huge.”

“Did it not approach the corpse?”

“No.”

“Was the night clear?”

“No; cloudy and humid.”

“But it wasn’t raining?”

“No.”

“What does the alley look like?”

“It is lined with two rows of yew trees and surrounded by a hedge, 12 feet high. The alley itself is 8 feet wide.”

“Is there anything between the hedge and the alley?”

“Yes; on both sides there is a lawn, 6 feet wide.”

“As I understand it, there is an entrance through the gate?”

“Yes; the gate leads to the meadow.”

“Are there other entrances?”

“None.”

“So, to enter Yew Alley, one must either leave the house or go through the meadow?”

“There is a third entrance through an arbor at the other end of the alley.”

“Did Sir Charles reach this arbor?”

“No; he was found fifty cubits from it.”

“And now, please tell me, Dr. Mortimer -- this will be an important detail -- were the tracks that you saw left on the gravel or on the grass?”

“I couldn’t see any tracks on the grass.”

“Were the tracks on the same side as the gate?”

“Yes, on the same side.”

“You intrigue me very much. One more question. Was the gate closed?”

“Locked and bolted.”

“How high is it?”

“About 4 feet.”

“So, it can be easily climbed over?”

“Yes.”

“And what tracks did you find near the gate?”

“None that might attract attention.”

“Did you not examine the ground around it?”

“Yes, I did.”

“And you didn’t notice any tracks?”

“They were very faint. Apparently, Sir Charles stood here for five to ten minutes.”

“How do you know that?”

“Because the ash from the cigar fell twice on the ground.”

“Excellent. We have found a colleague after our own heart. True, Watson? But what were these tracks?”

“Footprints on the gravel. I couldn’t see any other signs.”

Sherlock Holmes slapped his knee.

“Why wasn’t I there!” he exclaimed. “This is a very interesting case, offering ample scope for scientific expertise. Why couldn’t I read this gravel card before other feet erased it! Dr. Mortimer, that you didn’t inform me sooner! If we fail to shed light on the matter, all responsibility will fall on you.”

“I couldn’t summon you, Mr. Holmes, because in that case, I would have to reveal these facts to the whole world, and I have already said that I did not wish to do so. Besides… besides…”

“Why don’t you finish?”

“There are areas that are inaccessible even to the brightest and most experienced detectives…”

“You mean to say that this falls within the scope of supernatural things?”

“I do not assert that definitively.”

“But you think so deep down.”

“Since that tragedy, facts have come to my attention that contradict the general laws of nature.”

“For example?”

“I learn that before this catastrophe, several people saw an unusual creature in the meadow, similar to the evil spirit of the Baskervilles. Everyone says that it was a huge, transparent dog. I questioned these people -- one of them is a peasant, the other a blacksmith, the third a farmer. Their testimonies are identical. A superstitious fear has gripped the entire region. No one will cross the meadow at night.”

“And you, a man of science, believe in such fables?” Holmes exclaimed.

“So far, my research has encompassed the world of the senses; I have tried to fight disease, but I cannot fight evil spirits. Besides, you must admit that the footprints are material evidence. Hugo’s dog was also not a supernatural phenomenon since it could bite to death, and yet it had something diabolical in it.”

“I see that you have switched to the camp of spiritualists. But please tell me one more thing, Dr. Mortimer. If you lean towards such views, why did you come to seek my advice? You say in one breath that it is not worth conducting an investigation into the murder of Sir Charles, and at the same time, you ask me to take on the case.”

“I did not ask for that.”

“So how can I help you?”

“By advising me what to do with Sir Henry Baskerville, who arrives at Waterloo Station --” Dr. Mortimer looked at his watch -- “in an hour and a quarter,” he finished.

“Is he the heir?”

“Yes. After Sir Charles’s death, we sought information about this gentleman and learned that he has a farm in Canada. According to our information, he is a blameless young man. I am not speaking now as a doctor, but as the executor of Sir Charles’s will.”

“Are there no other candidates for the inheritance?”

“None. It could only be Roger Baskerville, the youngest of the three brothers, of whom Sir Charles was the eldest. The second brother, who died prematurely, was the father of this Henry. The third, Roger, was a prodigal son, a living spiritual likeness of old Hugo. He made so much trouble in England that he could no longer stay here, fled to Central America, and died in 1876 of yellow fever. Henry is the last of the Baskervilles. In an hour and five minutes, I am to meet him at Waterloo Station. I had a telegram notifying me that he arrived in Southampton today. And now, Mr. Holmes, tell me, what do you advise me to do?”

“Why shouldn’t Sir Henry return to the home of his fathers?”

“It seems the natural thing to do, and yet, if we consider that a sudden and violent death awaits each of the Baskervilles who resides there… I am sure that if Sir Charles could have spoken to me before the catastrophe, he would have forbidden me to bring the last of the family into that cursed residence. On the other hand, the well-being of the entire region depends on the heir residing at Baskerville Hall. All the work begun by Sir Charles will be undone if no one lives in the mansion. I am afraid that the good of this region will tempt me to be disloyal to Sir Henry, and therefore I ask you for advice.”

Holmes pondered for a long moment.

“Therefore,” he said, “according to your conviction, some unclean force threatens the Baskervilles in those parts. You believe in this implicitly, don’t you?”

“I am ready to assume that this is the case.”

“In that case, this unclean force can prey on Baskerville as well in London as in Devonshire. Satan whose action would be localized would not be dangerous.”

“You are trying to ridicule my fear, Mr. Holmes. But if you yourself had seen these supernatural things, you would not feel like joking. From your words, I gather that this young man is as safe in London as he is in Devonshire. He arrives in fifty minutes. What do you advise me to do?”

“I advise you to take a cab, call the dog that is howling behind the door, and go to Waterloo Station to meet Sir Henry Baskerville.”

“And then?”

“And then you will tell him nothing until I have made up my mind in this regard.”

“How long do you need to make up your mind?”

“Twenty-four hours. I will ask you, Dr. Mortimer, to visit me tomorrow morning at ten o'clock. And please bring Sir Henry Baskerville with you. This will make it easier for me to carry out my plan.”

“I will do as you wish.”

The doctor wrote down the hour on his cuff and left. Mr. Holmes stopped him on the stairs.

“One more question,” he said. “You say that before Sir Charles’s death, several people saw this phenomenon in the meadow?”

“Yes, three people.”

“Did any of them see it later?”

“I don’t know.”

“Thank you. Good-bye.”

Holmes returned to his seat. He was visibly pleased.

“Are you going out, Watson?” he said.

“Do you need my help?”

“No, my dear. I will only ask for it in the moment of action. An exceptional case. When passing Bradley’s, please stop by the store and have them bring me a package of the strongest tobacco. If you can, I would be very obliged if you did not return here before evening, and then we will compare our impressions and views.”

I knew that solitude was essential to my friend in moments when he was pondering the clues of criminal cases, when he was drawing conclusions and forming theories that always proved correct. So, I spent the whole day at the club and only returned to the apartment on Baker Street around nine o’clock.

When I opened the door, it seemed to me that there was a fire in the apartment; the light of the lamp appeared as if behind a black mist. After a moment, I realized that the smoke came not from fire, but from strong tobacco. Among the clouds, I saw Holmes in a dressing gown. He was sitting in an armchair with a pipe in his mouth. Several rolls of paper lay on the table. I coughed.

“Have you caught a cold?” he asked.

“No, but one can suffocate in here.”

“Open the window. I see that you spent the whole day at the club…”

“How do you know that, Holmes?”

“You are sober, fragrant, in a good mood. You will never guess where I was.”

“I won’t rack my brains over it. You will tell me yourself.”

“So, I was in Devonshire.”

“In thought?”

“Yes. My body remained here, in this armchair, and consumed two enormous pots of coffee and countless amounts of tobacco. After you left, I sent to Stanford for a map of this area and wandered around it all day. I flatter myself that every inch of land is now well known to me.”

“That is probably a large-scale map?”

“Yes.”

He unfolded it and placed it on his knee.

“You see,” he said, “here is the meadow, and this is Baskerville Hall.”

“Around it, the forest.”

“Indeed. And here is Yew Alley, to the left of the meadow. Here is the village of Grimpen, where Dr. Mortimer has taken his main quarters. Within five miles, there are few human dwellings. Here is Lafter Hall. Here -- the cottage of the naturalist Stapleton. Here are two forms: High Tore and Fulmire. Fourteen miles further -- the state prison of Princetown. Around and in the middle -- meadows and bogs. This is the terrain on which this tragedy took place. We will try to recreate all its scenes and acts.”

“A deserted place.”

“Yes. The devil could take up residence there…”

“So, you too are inclined to supernatural explanations…”


In [ ]:
translated_texts = [result[0] for result in results if isinstance(result, tuple)]

# join the results together by double whitespace
translated_book = "\n\n".join(translated_texts)

# save the translated book
with open(f"/content/translated_book.txt", "w") as f:
  f.write(translated_book)

## Next steps

Now you know how to properly prepare data, check token limitations and translate each chunk to obtain your results. Feel free to try this process with a book that interests you!